# Script to extract nodes and edges from the Million Song Dataset

### Import the required libraries

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
import findspark
findspark.init()
import pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

### Create a Spark Session

In [ ]:
# sc = pyspark.SparkContext(appName="dataset_creation")
# sqlContext = SQLContext(sc)
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("dataset_creation") \
    .getOrCreate()

In [ ]:
# sc.stop()
# spark.stop()

## Extract node data

### Read data

In [ ]:
# set the path to 'track_metadata.db' file here
track_metadata_db_path = "C:\\Users\\mgandhi39\\Downloads\\DVA Project\\Dataset\MSD\\track_metadata.db"

In [ ]:
conn = sqlite3.connect(track_metadata_db_path)
cursor = conn.cursor()

In [ ]:
query = 'SELECT name FROM sqlite_master WHERE type="table"'
cursor.execute(query)
print(cursor.fetchall())

[('songs',)]


In [ ]:
query = "SELECT * FROM songs"
pandas_df = pd.read_sql_query(query, conn)

In [ ]:
# trim the data to smaller size for testing
no_of_records = 1000000
trimmed_pandas_df = pandas_df[:no_of_records]

In [ ]:
# this step takes a while to run 
# TO DO: find a better way to read data from .db file to spark
spark_df = spark.createDataFrame(pandas_df)

C:\Spark\spark-3.3.2-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
C:\Spark\spark-3.3.2-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [ ]:
spark_df.limit(5).toPandas().head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0
3,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003,2168257,-1,0
4,TRMMMWA128F426B589,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Rene Ablaze Presents Winter Sessions,AREQDTE1269FB37231,,Der Mystic,514.29832,0.000000,0.000000,0,2264873,-1,0


### Group by artist_id and aggregate fields

In [ ]:
# artists2 = spark_df.groupby("artist_id").agg({F.count(spark_df["track_id"]).alias("total_songs"), F.avg(spark_df["duration"]).alias("avg_duration"), F.avg(spark_df["artist_hotttnesss"]).alias("avg_hotttness"), F.avg(spark_df["artist_familiarity"]).alias("avg_familiarity"), F.min(spark_df["year"]).alias("oldest_track_year"), F.max(spark_df["year"]).alias("latest_track_year")})
artists = spark_df.groupby("artist_id").agg({"track_id": "count", "duration": "avg", "artist_hotttnesss": "avg", "artist_familiarity": "avg"})
artists = artists.withColumnRenamed("avg(duration)", "avg_duration").withColumnRenamed("count(track_id)", "total_tracks").withColumnRenamed("avg(artist_familiarity)", "avg_familiarity").withColumnRenamed("avg(artist_hotttnesss)", "avg_hotttnesss")

In [ ]:
artists.toPandas().head()

,artist_id,avg_duration,total_tracks,avg_familiarity,avg_hotttnesss
0,AR8IQZ81187FB54AEE,259.807776,37,0.549361,0.473365
1,ARI9UV41187B98AB58,234.255540,49,0.496291,0.338061
2,ARA82JG1187FB36131,428.350584,91,0.618297,0.375375
3,ARRCJQD1187FB47609,233.837260,10,0.624650,0.516555
4,AR4VS2U1187B99AECC,185.687710,56,0.515832,0.431931


In [ ]:
artists.summary().show()

+-------+------------------+------------------+------------------+-------------------+-------------------+
|summary|         artist_id|      avg_duration|      total_tracks|    avg_familiarity|     avg_hotttnesss|
+-------+------------------+------------------+------------------+-------------------+-------------------+
|  count|             44745|             44745|             44745|              44745|              44745|
|   mean|              null| 264.3877490388256|22.348865795060902| 0.4916330159971349|0.32100221438480836|
| stddev|              null| 138.6299187942246| 23.89139346078063|0.15571915408223608|0.14612344200489397|
|    min|AR002UA1187B9A637D|           0.41751|                 1|               -1.0|               -1.0|
|    25%|              null|202.05306055555556|                 4|     0.422239845152|     0.282415437521|
|    50%|              null| 239.8637097999999|                14| 0.5028677373563002| 0.3536772562170001|
|    75%|              null|296.75056

### Add artist_name to the extracted data
An artist_id may correspond to more than one artist_name i.e. artist_name is not unique. This is because artist of a track can be 'Abc ft. Xyz'. So we keep the name which is of the shortest length.


In [ ]:
artist_names = spark_df.groupby(["artist_id", "artist_name"]).count()

In [ ]:
artist_names.show()

+------------------+--------------------+-----+
|         artist_id|         artist_name|count|
+------------------+--------------------+-----+
|ARX07G11187B9AC915|          Giant Drag|   26|
|ARJ0AL61187B9A3F27|         Katie Melua|   63|
|AR06M901187FB51091|   Nadine Sutherland|   46|
|ARUPVWK1187B992132|         Charanga 76|    1|
|ARPATQ21187B9ACAFC|       Happy Mondays|   89|
|AR8G5MM1187B9AA1D6|          Watchmaker|   49|
|AR817XQ1187FB3A654|     Patrik Isaksson|   50|
|ARYVIIA1187B9A2BD4|          Cold Sweat|   28|
|ARZM3YY1187B99BE41|Bill Nelson's Red...|   11|
|ARB2LGI1187FB4CD55|Archie Bronson Ou...|   41|
|ARRR9QU1187B9AE226|              Poison|   49|
|ARW0Z9V1187B9A0699|          Peter Tosh|   95|
|ARBRH4T1187B990D16|      The Envy Corps|   19|
|ARHFGDK1269FCD0ED6|    Daru & Reggie B.|   17|
|AR2TN021187B998B29|            Declaime|   38|
|AR7SPB81187B9A068F|          Stray Cats|   77|
|ARBUOOR1187B997391|               Oasis|  125|
|ARY77941187FB471BA|       Nenhum De Nós

In [ ]:
artist_names.summary().show()

+-------+------------------+--------------------+------------------+
|summary|         artist_id|         artist_name|             count|
+-------+------------------+--------------------+------------------+
|  count|             78236|               78236|             78236|
|   mean|              null|            Infinity|12.781839562349813|
| stddev|              null|                 NaN|19.478259684598253|
|    min|AR002UA1187B9A637D|Little Louie" Ve...|                 1|
|    25%|              null|               113.0|                 1|
|    50%|              null|               702.0|                 3|
|    75%|              null|              1929.0|                16|
|    max|ARZZZKG1271F573BC4|                 üNN|               194|
+-------+------------------+--------------------+------------------+



In [ ]:
window = Window.partitionBy(artist_names["artist_id"]).orderBy(length(artist_names["artist_name"]))
unique_artist_names = artist_names.select('*', row_number().over(window).alias('rank')).filter(col('rank') == 1)
unique_artist_names = unique_artist_names.drop("count").drop("rank").withColumnRenamed("artist_id", "id")

In [ ]:
unique_artist_names.limit(5).toPandas().head()

,id,artist_name
0,AR002UA1187B9A637D,The Bristols
1,AR003FB1187B994355,The Feds
2,AR006821187FB5192B,Stephen Varcoe
3,AR009211187B989185,Carroll Thompson
4,AR009SZ1187B9A73F4,Gorodisch


In [ ]:
unique_artist_names.count()

44745

In [ ]:
node_data = artists.join(unique_artist_names, artists["artist_id"] == unique_artist_names["id"], "inner").drop("id")

In [ ]:
node_data.limit(10).toPandas().head()

,artist_id,avg_duration,total_tracks,avg_familiarity,avg_hotttnesss,artist_name
0,AR002UA1187B9A637D,148.877188,9,0.450512,0.348651,The Bristols
1,AR003FB1187B994355,185.479987,26,0.576354,0.349057,The Feds
2,AR006821187FB5192B,173.543413,21,0.252652,0.328754,Stephen Varcoe
3,AR009211187B989185,265.496145,24,0.396818,0.297405,Carroll Thompson
4,AR009SZ1187B9A73F4,201.983547,10,0.489411,0.300847,Gorodisch


In [ ]:
# TO DO
# create links between colaborators

### Output node data to CSV

In [ ]:
# set nodes.csv output path here
node_data_output_path = "C:\\Users\\mgandhi39\\Downloads\\DVA Project\\Dataset\\nodes.csv"

In [ ]:
node_data.toPandas().to_csv(node_data_output_path)

## Extract edge data

### Read Data and extract edges

In [ ]:
# set the path to 'artist_similarity.db' file here
artist_similarity_db_path = "C:\\Users\\mgandhi39\\Downloads\\DVA Project\\Dataset\MSD\\artist_similarity.db"

In [ ]:
conn2 = sqlite3.connect(artist_similarity_db_path)
cursor2 = conn2.cursor()

In [ ]:
query = 'SELECT name FROM sqlite_master WHERE type="table"'
cursor2.execute(query)
print(cursor2.fetchall())

[('artists',), ('similarity',)]


In [ ]:
pd_edges = pd.read_sql_query('SELECT * FROM similarity', conn2)
pd_edges["index"] = range(1, len(pd_edges) + 1)

In [ ]:
spark_edges = spark.createDataFrame(pd_edges)
spark_edges = spark_edges.withColumnRenamed("target", "source_artist_id").withColumnRenamed("similar","target_artist_id")

C:\Spark\spark-3.3.2-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
C:\Spark\spark-3.3.2-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [ ]:
window2 = Window.partitionBy("source_artist_id").orderBy("index")
edge_data = spark_edges.select("*", row_number().over(window2).alias("priority")).drop("index")

In [ ]:
edge_data.limit(5).toPandas().head()

,source_artist_id,target_artist_id,priority
0,AR002UA1187B9A637D,ARQDOR81187FB3B06C,1
1,AR002UA1187B9A637D,AROHMXJ1187B989023,2
2,AR002UA1187B9A637D,ARAGWVR1187B9B749B,3
3,AR002UA1187B9A637D,AREQVWS1241B9CC0A4,4
4,AR002UA1187B9A637D,ARHBE351187FB3B0CD,5


In [ ]:
edge_data.summary().show()

+-------+------------------+------------------+------------------+
|summary|  source_artist_id|  target_artist_id|          priority|
+-------+------------------+------------------+------------------+
|  count|           2201916|           2201916|           2201916|
|   mean|              null|              null| 29.09958145542337|
| stddev|              null|              null|19.068475148295683|
|    min|AR002UA1187B9A637D|AR002UA1187B9A637D|                 1|
|    25%|              null|              null|                13|
|    50%|              null|              null|                27|
|    75%|              null|              null|                43|
|    max|ARZZZKG1271F573BC4|ARZZZKG1271F573BC4|                97|
+-------+------------------+------------------+------------------+



### Output edge data to CSV

In [ ]:
# set edges.csv output path here
edge_data_output_path = "C:\\Users\\mgandhi39\\Downloads\\DVA Project\\Dataset\edges.csv"

In [ ]:
edge_data.toPandas().to_csv(edge_data_output_path)